<a href="https://colab.research.google.com/github/DerickW126/Fake-News-Project/blob/main/Clickbait.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 59.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 75.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from scipy.sparse import coo_matrix
from scipy.sparse.linalg import svds
from scipy.sparse import csr_matrix
import spacy

nlp = spacy.load("en_core_web_sm")
from wordcloud import WordCloud

from tabulate import tabulate
from tabulate import tabulate
from tqdm import trange
import random

import datetime as d
import math
import re
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
import pylab
warnings.filterwarnings('ignore')

%matplotlib inline

import glob
from google.colab import drive
import os
glob.glob("*.csv.zip")
drive.mount('/content/drive')
%cd /content/drive/MyDrive
os.system("unzip Clickbait.zip")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive


2304

In [ ]:
import glob
from google.colab import drive
import os
glob.glob("*.csv.zip")
drive.mount('/content/drive')
%cd /content/drive/MyDrive

Mounted at /content/drive
/content/drive/MyDrive


In [ ]:
df = pd.read_csv("clickbait_data.csv")
headlines = df['headline']

# Create a list of tokens for each headline
docs = [nlp(headline) for headline in headlines]

# Create a list of words for each headline
words = [[token.text for token in doc] for doc in docs]
# Join the words for each headline into a single string
text = [' '.join(headline) for headline in words]

# Create a WordCloud object with the text
wordcloud = WordCloud(width=800, height=800, background_color='white').generate(' '.join(text))

In [ ]:
vectorizer = TfidfVectorizer(stop_words='english')

# Fit the vectorizer to the data
vectorizer.fit(df['headline'])

# Transform the data into a tf-idf matrix
tfidf = vectorizer.transform(df['headline'])


In [ ]:
text = df["headline"][100]
re.findall(r"\b(who|what|where|when|why|how|should|which|do)\b", text, re.IGNORECASE) is not []

True

In [ ]:
def features_clickbait(df):
    
    cols = ['Headline','Sensational words','Ambiguous','Misleading','NumbersLists','Emotional appeals','Exaggeration','Teaser','Superlatives','Celebrity','Incomplete','Punctuation','ImagesVideos','Question','Personal pronouns','Urgency','Hyperbole','Offensive words','Clickbait']
    df_clickbait = pd.DataFrame(columns=cols)
    
    for i in range(df.shape[0]):
        
        text = df["headline"][i]
        
        features = [df["headline"][i]]
    
        # 1. Check for sensational language
        sensational_words = ['shocking', 'outrageous', 'amazing', 'unbelievable', 'mind-blowing']
        count = 0
        for word in sensational_words:
            if word in text.lower():
                count += 1
        features.append(count)

        # 2. Check for ambiguous or vague headlines
        exist = 0
        if '?' in text:
            exist = 1
        features.append(exist)
            

        # 3. Check for misleading or false claims
        count = 0
        if re.search(r"\bnot clickbait\b", text, re.IGNORECASE) is not None:
            count += len(re.findall(r"\bnot clickbait\b", text, re.IGNORECASE))
        if re.search(r"\b(click|you won't believe|must see)\b", text, re.IGNORECASE) is not None:
            count += len(re.findall(r"\b(click|you won't believe|must see)\b", text, re.IGNORECASE))
        features.append(count)

        # 4. Check for use of numbers and lists
        count = 0
        if re.search(r"\b\d+ (reasons|things|ways|facts)\b", text, re.IGNORECASE) is not None:
            count += len(re.findall(r"\b\d+ (reasons|things|ways|facts)\b", text, re.IGNORECASE))
        features.append(count)

        # 5. Check for emotional appeals
        emotional_words = ['fear', 'outrage', 'anger', 'hate', 'heartbreaking', 'tragic']
        count = 0
        for word in emotional_words:
            if word in text.lower():
                count += 1
        features.append(count)

        # 6. Check for exaggerated claims
        count = 0
        if re.search(r"\b(never seen|world's best|ultimate guide|perfect solution)\b", text, re.IGNORECASE) is not None:
            count += len(re.findall(r"\b(never seen|world's best|ultimate guide|perfect solution)\b", text, re.IGNORECASE))
        features.append(count)

        # 7. Check for teaser text
        features.append(len(text))


        # 8. Check for use of superlatives
        count = 0
        if re.search(r"\b(best|worst|most|greatest)\b", text, re.IGNORECASE) is not None:
            count += len(re.findall(r"\b(best|worst|most|greatest)\b", text, re.IGNORECASE))
        features.append(count)

        # 9. Check for use of celebrity names
        count = 0
        if re.search(r"\b(Kardashian|Bieber|Swift|'Pitt'|Jolie|Cruise)\b", text, re.IGNORECASE) is not None:
            count += len(re.findall(r"\b(Kardashian|Bieber|Swift|'Pitt'|Jolie|Cruise)\b", text, re.IGNORECASE))
        features.append(count)

        # 10. Check for incomplete headlines
        features.append(len(text.split()))

        # 11. Check for excessive punctuation
        count = 0
        if re.search(r"\b(!+|\?+)$", text) is not None:
            count += len(re.findall(r"\b(!+|\?+)$", text))
        features.append(count)

        # 12. Check for use of images or videos
        count = 0
        if re.search(r"\b(picture|photo|video)\b", text, re.IGNORECASE) is not None:
            count += len(re.findall(r"\b(picture|photo|video)\b", text, re.IGNORECASE))
        features.append(count)

        # 13. Check for headlines that ask a question
        count = 0
        if re.search(r"\b(who|what|where|when|why|how|should|which|do)\b", text, re.IGNORECASE) is not None:
            count += len(re.findall(r"\b(who|what|where|when|why|how|should|which|do)\b", text, re.IGNORECASE))
        features.append(count)

        # 14. Check for use of personal pronouns
        count = 0
        if re.search(r"\b(i|you|your)\b", text, re.IGNORECASE) is not None:
            count += len(re.findall(r"\b(i|you|your)\b", text, re.IGNORECASE))
        features.append(count)
        
        # 15. Use of urgency
        count = 0
        urgent_words = ['limited time', 'only', 'last chance', 'hurry', 'act now']
        for word in urgent_words:
            if word in text.lower():
                count += 1
        features.append(count)

        # 16. Use of hyperbole
        count = 0
        hyperbole_words = ['epic', 'mind-bending', 'life-changing', 'jaw-dropping', 'unforgettable']
        for word in hyperbole_words:
            if word in text.lower():
                count += 1
        features.append(count)


        # 17. Use of provocative or offensive language
        count = 0 
        offensive_words = ['hate', 'shocking', 'outrageous', 'disgusting', 'repulsive']
        for word in offensive_words:
            if word in text.lower():
                count += 1
        features.append(count)
                
        features.append(df["clickbait"][i])
        
        
        df_clickbait.loc[i] = features
    
    return df_clickbait

In [ ]:
df_clickbait = features_clickbait(df)

In [ ]:
text = df.headline.values
labels = df.clickbait.values
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',do_lower_case = True)

In [ ]:
token_id = []
attention_masks = []

def preprocessing(input_text, tokenizer):
  '''
  Returns <class transformers.tokenization_utils_base.BatchEncoding> with the following fields:
    - input_ids: list of token ids
    - token_type_ids: list of token type ids
    - attention_mask: list of indices (0,1) specifying which tokens should considered by the model (return_attention_mask = True).
  '''
  return tokenizer.encode_plus(
                        input_text,
                        add_special_tokens = True,
                        max_length = 32,
                        pad_to_max_length = True,
                        return_attention_mask = True,
                        return_tensors = 'pt'
                   )


for sample in text:
  encoding_dict = preprocessing(sample, tokenizer)
  token_id.append(encoding_dict['input_ids']) 
  attention_masks.append(encoding_dict['attention_mask'])


token_id = torch.cat(token_id, dim = 0)
attention_masks = torch.cat(attention_masks, dim = 0)
labels = torch.tensor(labels)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [ ]:
val_ratio = 0.2 #20% validation

batch_size = 16

# Indices of the train and validation splits stratified by labels
train_idx, val_idx = train_test_split(
    np.arange(len(labels)),
    test_size = val_ratio,
    shuffle = True,
    stratify = labels)

# Train and validation sets
train_set = TensorDataset(token_id[train_idx], 
                          attention_masks[train_idx], 
                          labels[train_idx])

val_set = TensorDataset(token_id[val_idx], 
                        attention_masks[val_idx], 
                        labels[val_idx])

# Prepare DataLoader
train_dataloader = DataLoader(
            train_set,
            sampler = RandomSampler(train_set),
            batch_size = batch_size
        )

validation_dataloader = DataLoader(
            val_set,
            sampler = SequentialSampler(val_set),
            batch_size = batch_size
        )

In [ ]:
def b_tp(preds, labels):
  '''Returns True Positives (TP): count of correct predictions of actual class 1'''
  return sum([preds == labels and preds == 1 for preds, labels in zip(preds, labels)])

def b_fp(preds, labels):
  '''Returns False Positives (FP): count of wrong predictions of actual class 1'''
  return sum([preds != labels and preds == 1 for preds, labels in zip(preds, labels)])

def b_tn(preds, labels):
  '''Returns True Negatives (TN): count of correct predictions of actual class 0'''
  return sum([preds == labels and preds == 0 for preds, labels in zip(preds, labels)])

def b_fn(preds, labels):
  '''Returns False Negatives (FN): count of wrong predictions of actual class 0'''
  return sum([preds != labels and preds == 0 for preds, labels in zip(preds, labels)])

def b_metrics(preds, labels):
  '''
  Returns the following metrics:
    - accuracy    = (TP + TN) / N
    - precision   = TP / (TP + FP)
    - recall      = TP / (TP + FN)
    - specificity = TN / (TN + FP)
  '''
  preds = np.argmax(preds, axis = 1).flatten()
  labels = labels.flatten()
  tp = b_tp(preds, labels)
  tn = b_tn(preds, labels)
  fp = b_fp(preds, labels)
  fn = b_fn(preds, labels)
  b_accuracy = (tp + tn) / len(labels)
  b_precision = tp / (tp + fp) if (tp + fp) > 0 else 'nan'
  b_recall = tp / (tp + fn) if (tp + fn) > 0 else 'nan'
  b_specificity = tn / (tn + fp) if (tn + fp) > 0 else 'nan'
  return b_accuracy, b_precision, b_recall, b_specificity

In [ ]:
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels = 2,
    output_attentions = False,
    output_hidden_states = False,
)

# Recommended learning rates (Adam): 5e-5, 3e-5, 2e-5. See: https://arxiv.org/pdf/1810.04805.pdf
optimizer = torch.optim.AdamW(model.parameters(), 
                              lr = 5e-5,
                              eps = 1e-08
                              )

# Run on GPU
model.cuda()

In [ ]:
if torch.cuda.is_available():
    device = torch.device('cuda')
    
else:
    device = torch.device('cpu')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Recommended number of epochs: 2, 3, 4. See: https://arxiv.org/pdf/1810.04805.pdf
epochs = 2

for _ in trange(epochs, desc = 'Epoch'):
    
    # ========== Training ==========
    
    # Set model to training mode
    model.train()
    
    # Tracking variables
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0

    for step, batch in enumerate(train_dataloader):
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        optimizer.zero_grad()
        # Forward pass
        train_output = model(b_input_ids, 
                             token_type_ids = None, 
                             attention_mask = b_input_mask, 
                             labels = b_labels)
        # Backward pass
        train_output.loss.backward()
        optimizer.step()
        # Update tracking variables
        tr_loss += train_output.loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1

    # ========== Validation ==========

    # Set model to evaluation mode
    model.eval()

    # Tracking variables 
    val_accuracy = []
    val_precision = []
    val_recall = []
    val_specificity = []

    for batch in validation_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        with torch.no_grad():
          # Forward pass
          eval_output = model(b_input_ids, 
                              token_type_ids = None, 
                              attention_mask = b_input_mask)
        logits = eval_output.logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        # Calculate validation metrics
        b_accuracy, b_precision, b_recall, b_specificity = b_metrics(logits, label_ids)
        val_accuracy.append(b_accuracy)
        # Update precision only when (tp + fp) !=0; ignore nan
        if b_precision != 'nan': val_precision.append(b_precision)
        # Update recall only when (tp + fn) !=0; ignore nan
        if b_recall != 'nan': val_recall.append(b_recall)
        # Update specificity only when (tn + fp) !=0; ignore nan
        if b_specificity != 'nan': val_specificity.append(b_specificity)

    print('\n\t - Train loss: {:.4f}'.format(tr_loss / nb_tr_steps))
    print('\t - Validation Accuracy: {:.4f}'.format(sum(val_accuracy)/len(val_accuracy)))
    print('\t - Validation Precision: {:.4f}'.format(sum(val_precision)/len(val_precision)) if len(val_precision)>0 else '\t - Validation Precision: NaN')
    print('\t - Validation Recall: {:.4f}'.format(sum(val_recall)/len(val_recall)) if len(val_recall)>0 else '\t - Validation Recall: NaN')
    print('\t - Validation Specificity: {:.4f}\n'.format(sum(val_specificity)/len(val_specificity)) if len(val_specificity)>0 else '\t - Validation Specificity: NaN')

Epoch:  50%|█████     | 1/2 [03:30<03:30, 210.49s/it]


	 - Train loss: 0.0625
	 - Validation Accuracy: 0.9889
	 - Validation Precision: 0.9912
	 - Validation Recall: 0.9881
	 - Validation Specificity: 0.9898



Epoch: 100%|██████████| 2/2 [06:57<00:00, 208.92s/it]


	 - Train loss: 0.0207
	 - Validation Accuracy: 0.9850
	 - Validation Precision: 0.9883
	 - Validation Recall: 0.9824
	 - Validation Specificity: 0.9874



In [ ]:
new_sentence = 'Click here to see the latest news about Dua Lipa'
load_model.to(device)

# We need Token IDs and Attention Mask for inference on the new sentence
test_ids = []
test_attention_mask = []

# Apply the tokenizer
encoding = preprocessing(new_sentence, tokenizer)

# Extract IDs and Attention Mask
test_ids.append(encoding['input_ids'])
test_attention_mask.append(encoding['attention_mask'])
test_ids = torch.cat(test_ids, dim = 0)
test_attention_mask = torch.cat(test_attention_mask, dim = 0)

# Forward pass, calculate logit predictions
with torch.no_grad():
  output = load_model(test_ids.to(device), token_type_ids = None, attention_mask = test_attention_mask.to(device))

prediction = 'Clickbait' if np.argmax(output.logits.cpu().numpy()).flatten().item() == 1 else 'Not_clickbait'

print('Input Sentence: ', new_sentence)
print('Predicted Class: ', prediction)

Input Sentence:  Click here to see the latest news about Dua Lipa
Predicted Class:  Clickbait


In [ ]:
model.save_pretrained('Clickbait_model')

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification

load_model = BertForSequenceClassification.from_pretrained('Clickbait_model')

In [ ]:
def preprocessing(input_text, tokenizer):
  '''
  Returns <class transformers.tokenization_utils_base.BatchEncoding> with the following fields:
    - input_ids: list of token ids
    - token_type_ids: list of token type ids
    - attention_mask: list of indices (0,1) specifying which tokens should considered by the model (return_attention_mask = True).
  '''
  return tokenizer.encode_plus(
                        input_text,
                        add_special_tokens = True,
                        max_length = 32,
                        pad_to_max_length = True,
                        return_attention_mask = True,
                        return_tensors = 'pt'
                   )

In [ ]:
import torch
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',do_lower_case = True)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
new_sentence = 'Click here to see the latest news about Dua Lipa'
load_model.to(device)

# We need Token IDs and Attention Mask for inference on the new sentence
test_ids = []
test_attention_mask = []

# Apply the tokenizer
encoding = preprocessing(new_sentence, tokenizer)

# Extract IDs and Attention Mask
test_ids.append(encoding['input_ids'])
test_attention_mask.append(encoding['attention_mask'])
test_ids = torch.cat(test_ids, dim = 0)
test_attention_mask = torch.cat(test_attention_mask, dim = 0)

# Forward pass, calculate logit predictions
with torch.no_grad():
  output = load_model(test_ids.to(device), token_type_ids = None, attention_mask = test_attention_mask.to(device))

prediction = 'Clickbait' if np.argmax(output.logits.cpu().numpy()).flatten().item() == 1 else 'Not_clickbait'

print('Input Sentence: ', new_sentence)
print('Predicted Class: ', prediction)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Input Sentence:  Click here to see the latest news about Dua Lipa
Predicted Class:  Clickbait
